# Clase 3 — Desarrollo de aplicaciones con Visión por Computador (3 horas)

---

## Objetivos de aprendizaje

Al finalizar la sesión, los estudiantes serán capaces de:

1. Diseñar una API REST segura y eficiente para servir modelos de visión por computador.
2. Implementar endpoints para clasificación y detección que sigan buenas prácticas (validación, manejo de errores, tipado).
3. Exportar un modelo PyTorch a TorchScript u ONNX y explicar las ventajas de cada formato para producción.
4. Contenerizar la aplicación con Docker y describir consideraciones para despliegue en producción.

---



## Requisitos previos y recursos

* Python 3.9+
* PyTorch + torchvision
* FastAPI, Uvicorn
* Docker (para la parte de contenerización)

Ejemplo mínimo de `requirements.txt`:

```
fastapi>=0.95
uvicorn[standard]>=0.20
torch>=1.12
torchvision>=0.13
pydantic>=1.10
numpy>=1.23
python-multipart>=0.0.5
pytest>=7.0
requests>=2.28
```

---



## Parte teórica

### 1) ¿Por qué separar modelo y aplicación?

* **Separación de responsabilidades**: modelo (inferencia) vs API (validación, seguridad, logging).
* **Escalabilidad**: independizar el servicio permite escalar el modelo por separado (más réplicas o GPU nodes).
* **Observabilidad**: el servicio web permite instrumentación (métricas, logs, traces).

### 2) Opciones de serving

* **FastAPI**: rápido de desarrollar, documentación automática (OpenAPI), ideal para PoC y microservicios. Permite endpoints sin complicaciones.
* **TorchServe**: diseñado para modelos PyTorch en producción; soporta batching, métricas y gestion de endpoints con modelos múltiples.
* **TensorFlow Serving / TF-Serving**: equivalente para modelos TensorFlow.
* **ONNX Runtime**: para ejecutar modelos exportados a ONNX con alto rendimiento en CPU y aceleradores.

**Regla**: para prototipos y clases use FastAPI; para producción con necesidades de escalado/batching considere TorchServe/ONNX.

### 3) Requisitos de diseño de API

* Endpoints claros: `/health`, `/predict/classify`, `/predict/detect`.
* Validación de entrada con Pydantic (tamaños máximos, tipos, formatos MIME permitidos).
* Soporte para subida de archivos (`multipart/form-data`) y payloads base64.
* Respuestas JSON estandarizadas (clave `status`, `predictions`, `error`, `meta`).

### 4) Seguridad y privacidad mínima

* Limitar tamaño de subida (`max_content_length`), evitar DoS por archivos grandes.
* Autenticación: JWT / API keys para acceso a la API en producción.
* Si se procesan imágenes con datos personales: anonimización, políticas de retención y cumplimiento (GDPR/HIPAA según correspondencia).

---



## Laboratorio práctico — Integración de una CNN en un servicio web

### Objetivo práctico

Construir un servicio web (FastAPI) que:

1. Cargue un modelo PyTorch (preentrenado o fine-tuned).
2. Exponga un endpoint `/predict/classify` que acepte una imagen y devuelva la clase y probabilidades.
3. (Opcional) Exponga `/predict/detect` que devuelva bounding boxes y scores usando un detector preentrenado.

### Estructura de archivos sugerida

```
cv-app/
├── app/
│   ├── main.py            # FastAPI app
│   ├── model.py           # carga, preproc y postproc del modelo
│   └── schemas.py         # Pydantic models para requests/responses
├── requirements.txt
```

> Recomendación: mantenga la lógica de preprocesado y postprocesado en `model.py` para facilitar tests y posible exportación.

### Código: `app/model.py` (ejemplo para clasificación con ResNet18)



In [ ]:
# app/model.py
from __future__ import annotations

import io
import logging
from typing import List, Tuple
import json

import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision import models

logger = logging.getLogger("cv_app.model")

class ImageClassifier:
    def __init__(self, device: str = "cpu") -> None:
        self.device = device
        self.model = self._load_model()
        self.model.to(self.device)
        self.model.eval()
        self.labels = self._load_imagenet_labels()
        # Imagenet mean/std para pre-trained models (optimizado para MobileNet)
        self.transform = T.Compose([
            T.Resize(224),  # Reducido de 256 a 224 directamente
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def _load_model(self) -> torch.nn.Module:
        logger.info("Descargando modelo MobileNet (ligero)...")
        model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1)
        logger.info("Modelo MobileNet descargado y cargado")
        # si tiene que cambiar la cabeza para num_classes != 1000, hacerlo aquí
        return model

    def _load_imagenet_labels(self) -> List[str]:
        """Carga las etiquetas de ImageNet de forma simple"""
        try:
            # Usar las etiquetas que vienen con el modelo (más confiable)
            from torchvision.models import get_model_weights
            weights = get_model_weights('mobilenet_v3_small')
            labels = weights.IMAGENET1K_V1.meta['categories']
            logger.info(f"Cargadas {len(labels)} etiquetas desde weights")
            return labels
        except Exception as e:
            logger.warning(f"No se pudieron cargar las etiquetas desde weights: {e}")
            # Fallback: etiquetas genéricas más descriptivas
            return [f"objeto_{i}" for i in range(1000)]

    def preprocess(self, image_bytes: bytes) -> torch.Tensor:
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        x = self.transform(image).unsqueeze(0)  # batch dimension
        return x.to(self.device)

    @torch.inference_mode()
    def predict(self, image_bytes: bytes, topk: int = 5) -> List[Tuple[str, float]]:
        x = self.preprocess(image_bytes)
        logits = self.model(x)
        probs = torch.nn.functional.softmax(logits, dim=1)
        top_probs, top_idxs = probs.topk(topk, dim=1)
        top_probs = top_probs.cpu().numpy().flatten().tolist()
        top_idxs = top_idxs.cpu().numpy().flatten().tolist()
        # Mapear índices a etiquetas reales de ImageNet
        labels = [self.labels[idx] for idx in top_idxs]
        return list(zip(labels, top_probs))


### Código: `app/schemas.py` (Pydantic)



In [ ]:
# app/schemas.py
from pydantic import BaseModel, Field
from typing import List, Tuple

class Prediction(BaseModel):
    label: str
    score: float

class ClassifyResponse(BaseModel):
    status: str = Field("ok")
    predictions: List[Prediction]


### Código: `app/main.py` (FastAPI app)



In [ ]:
# app/main.py
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
import logging

from model.model import ImageClassifier
from schemas.schema import ClassifyResponse, Prediction

logger = logging.getLogger("cv_app")
app = FastAPI(title="CV Model API", version="0.1")

# cargamos el modelo de forma lazy (cuando se necesite)
classifier: ImageClassifier | None = None

def get_classifier():
    global classifier
    if classifier is None:
        logger.info("Cargando modelo por primera vez...")
        classifier = ImageClassifier(device="cpu")
        logger.info("Modelo cargado exitosamente")
    return classifier


@app.get("/")
def root():
    """Endpoint raíz con información de la API"""
    return {
        "message": "API de Clasificación de Imágenes con IA",
        "version": "1.0.0",
        "description": "Clasifica imágenes usando MobileNet pre-entrenado en ImageNet",
        "endpoints": {
            "health": "/health - Estado del servidor",
            "classify": "/predict/classify - Clasificar una imagen",
            "test_labels": "/test-labels - Ver etiquetas del modelo",
            "docs": "/docs - Documentación interactiva"
        },
        "usage": {
            "method": "POST",
            "endpoint": "/predict/classify",
            "content_type": "multipart/form-data",
            "parameter": "file (imagen)"
        },
        "example": "Sube una imagen a /predict/classify para obtener las 3 predicciones más probables"
    }

@app.get("/health")
def health():
    return {"status": "ok"}

@app.get("/test-labels")
def test_labels():
    """Endpoint para probar las etiquetas del modelo"""
    classifier = get_classifier()
    # Mostrar las primeras 10 etiquetas
    sample_labels = classifier.labels[:10]
    return {"labels_sample": sample_labels, "total_labels": len(classifier.labels)}

@app.post("/predict/classify", response_model=ClassifyResponse)
async def predict_classify(file: UploadFile = File(...)):
    if not file.content_type.startswith("image/"):
        raise HTTPException(status_code=400, detail="Archivo no es una imagen")
    image_bytes = await file.read()
    if len(image_bytes) == 0:
        raise HTTPException(status_code=400, detail="Archivo vacío")
    classifier = get_classifier()  # Carga el modelo solo cuando se necesita
    preds = classifier.predict(image_bytes, topk=3)  # Reducido de 5 a 3 predicciones
    response = ClassifyResponse(predictions=[Prediction(label=p[0], score=p[1]) for p in preds])
    return response


# Ahora...

## Crear la carpeta /img y guardar las imagenes para probar el modelo

## Ejecutar la aplicación
```
uvicorn main:app --reload --host 127.0.0.1 --port 8000
```
### **2. En el navegador web:**
- **Documentación**: `http://127.0.0.1:8000/docs`
- **API principal**: `http://127.0.0.1:8000`
- **Health check**: `http://127.0.0.1:8000/health`

### **3. Para probar la clasificación de imágenes:**

1. **Abre tu navegador**
2. **Ve a**: `http://127.0.0.1:8000/docs`
3. **Verás la interfaz de Swagger** con los endpoints
4. **Haz clic en** `/predict/classify`
5. **Haz clic en** "Try it out"
6. **Sube una imagen** y haz clic en "Execute"

### **4. La consola mostrará:**
- Cuando alguien haga una petición
- Los logs de carga del modelo (primera vez)
- Los logs de procesamiento de imágenes



## Lecturas y recursos

* FastAPI docs (Auto-generated OpenAPI).
* PyTorch/TorchVision model zoo.
* TorchServe docs y ejemplos para serving en producción.
* ONNX Runtime docs.

